In [3]:
! pip install mlxtend

  Using cached mlxtend-0.21.0-py2.py3-none-any.whl (1.3 MB)


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import MinMaxScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from mlxtend.feature_selection import SequentialFeatureSelector as SFS


In [5]:
df_train=pd.read_csv("house_price_train.csv")
df_train.drop("Id",axis=1,inplace=True) ## reduntatnt
output_col=df_train['SalePrice']
df_train=df_train.drop('SalePrice',axis=1)

In [6]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


In [7]:
df_train.dtypes

MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MiscVal            int64
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
Length: 79, dtype: object

## PREPROCESSING


In [8]:
# df_train.columns[df_train.isnull().any()]

In [9]:
# sns.heatmap(df_train.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [10]:
# plt.figure(figsize=(30,30))
# sns.heatmap(pd.concat([df_train,output_col],axis=1).loc[:,list(pd.concat([df_train,output_col],axis=1).select_dtypes(include= np.number).columns)].corr(method='spearman'),annot=True,cmap="RdYlGn")
# plt.show()

In [11]:
# fig, axes = plt.subplots(nrows = 12, ncols = 3)    # axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(30,30)
# cols=list(df_train.select_dtypes(include= np.number).columns)
# for ax, col in zip(axes, cols):
#     print()
#     sns.boxplot(df_train[col], ax = ax)
#     ax.set_title(col)

In [12]:
# fig, axes = plt.subplots(nrows = 12, ncols = 3)    # axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(30,30)
# cols=list(df_train.select_dtypes(include= np.number).columns)
# for ax, col in zip(axes, cols):
#     print()
#     sns.distplot(df_train[col], ax = ax)
#     ax.set_title(col)

In [13]:
def column_with_most_reduntant_null_values(threshold,df_train:pd.DataFrame):
    null_cols=[]
    for col in df_train.columns:
        if df_train[col].isnull().sum()>=(threshold*len(df_train)):
            null_cols.append(col)
    df_train.drop(null_cols,axis=1,inplace=True)

In [14]:
column_with_most_reduntant_null_values(threshold=0.5,df_train=df_train)

In [15]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,61,0,0,0,0,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,42,0,0,0,0,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,35,272,0,0,0,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,84,0,0,0,0,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,40,0,0,0,0,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,60,0,0,0,0,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,112,0,0,0,0,4,2010,WD,Normal


In [16]:
continuous_cols=list(df_train.select_dtypes(include= np.number).columns)

In [17]:
categorical_cols=list(df_train.select_dtypes(exclude= np.number).columns)

In [18]:
most_correlated=dict()
for col1 in continuous_cols:
    for col2 in continuous_cols:
        if col1!=col2 and abs(df_train[col1].corr(df_train[col2],method='spearman'))>=0.80:
            if (col1 not in most_correlated) and (col2 not in most_correlated):
                most_correlated[col1]=col2

In [19]:
df_train.drop(columns=list(most_correlated),axis=1,inplace=True)

In [20]:
def fill_null_values(for_numerical_cols:str,for_categorical_cols:str,df_train):
    for cols in df_train.columns:
        if df_train[cols].dtype=="int64" or df_train[cols].dtype=="float64":
            if for_numerical_cols=="median":
                median_=df_train[cols].median()
                df_train[cols].fillna(median_,inplace=True)
        elif df_train[cols].dtype=="object":
            if for_categorical_cols=="most_frequent":
                d=list(df_train[cols].value_counts().index) # most frequent
                df_train[cols].fillna(d[0],inplace=True)


In [21]:
fill_null_values(for_numerical_cols="median",for_categorical_cols="most_frequent",df_train=df_train)

In [22]:
for col in df_train.columns:
    if df_train[col].isnull().sum()>(0):

        print(col,df[col].isnull().sum(),df_train[col].dtypes)


In [23]:
continuous_cols=list(df_train.select_dtypes(include= np.number).columns)

In [24]:
categorical_cols=list(df_train.select_dtypes(exclude= np.number).columns)

We are encoding categorical columns using label encoder as 0,1,2

In [25]:

def encoding_categorical_cols(df_train:pd.DataFrame,categorical_columns:list,strategy:str):
    df_categorical=pd.DataFrame(df_train.loc[:,categorical_columns])
    if strategy=="labelencoder":
        for col in list(df_categorical.columns):
            cat=[]
            d={}

            cat.append([df_categorical[col].value_counts().index,len(df_categorical[col].value_counts().index)])

            for j in range(int(cat[0][1])):
                d[str(cat[0][0][j])]=j
            df_categorical[col]=df_categorical[col].map(d)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        
        df_train=pd.concat([df_train,df_categorical],axis=1)
        
    if strategy=="onehotencoder":
        one_hot_encoded=pd.get_dummies(df_categorical)
        # print(one_hot_encoded)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        df_train=pd.concat([df_train,one_hot_encoded],axis=1)
    return df_train

We have encoded our categorical columns using labelencoder. The reason we haven't used onehotencoder is explained below:
Suppose the categorical column "LotShape" have 5 categories then onehotencoder will generate 5 new columns . Now when we know Sequential Feature Selector(SFS) selects some column which gives us highest accuracy, now if SFS selects 4 columns(categories) of one encoded "LotShape" then the training accuracy might give us good results but when test or when we have to predict new datapoint then our accuracy will be vey low. 

In [26]:
df_train=encoding_categorical_cols(df_train=df_train,categorical_columns=categorical_cols,strategy="labelencoder")

In [27]:
df_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,60,65.0,8450,7,5,2003,196.0,706,0,150,...,1,0,0,0,1,0,0,0,0,0
1,20,80.0,9600,6,8,1976,0.0,978,0,284,...,0,0,1,0,1,0,0,0,0,0
2,60,68.0,11250,7,5,2002,162.0,486,0,434,...,1,0,1,0,1,0,0,0,0,0
3,70,60.0,9550,7,5,1970,0.0,216,0,540,...,1,0,0,1,0,0,0,0,0,2
4,60,84.0,14260,8,5,2000,350.0,655,0,490,...,1,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5,2000,0.0,0,0,953,...,0,0,1,0,1,0,0,0,0,0
1456,20,85.0,13175,6,6,1988,119.0,790,163,589,...,0,2,1,0,0,0,0,0,0,0
1457,70,66.0,9042,7,9,2006,0.0,275,0,877,...,1,0,0,0,1,0,0,0,0,0
1458,20,68.0,9717,5,6,1996,0.0,49,1029,0,...,1,0,0,0,0,0,0,0,0,0


In [28]:
for col in df_train.columns:
    if df_train[col].isnull().sum()>0:
        print(col)

Using Min Max scaler to make values in range 0 to 1.

In [29]:
scaler=scaler=MinMaxScaler()
df_train=pd.DataFrame(scaler.fit_transform(df_train),columns=df_train.columns)
df_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,0.235294,0.150685,0.033420,0.666667,0.500,0.883333,0.122500,0.125089,0.000000,0.064212,...,0.333333,0.000000,0.00,0.0,0.5,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.202055,0.038795,0.555556,0.875,0.433333,0.000000,0.173281,0.000000,0.121575,...,0.000000,0.000000,0.25,0.0,0.5,0.0,0.0,0.0,0.0,0.0
2,0.235294,0.160959,0.046507,0.666667,0.500,0.866667,0.101250,0.086109,0.000000,0.185788,...,0.333333,0.000000,0.25,0.0,0.5,0.0,0.0,0.0,0.0,0.0
3,0.294118,0.133562,0.038561,0.666667,0.500,0.333333,0.000000,0.038271,0.000000,0.231164,...,0.333333,0.000000,0.00,0.2,0.0,0.0,0.0,0.0,0.0,0.4
4,0.235294,0.215753,0.060576,0.777778,0.500,0.833333,0.218750,0.116052,0.000000,0.209760,...,0.333333,0.000000,0.25,0.0,0.5,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.235294,0.140411,0.030929,0.555556,0.500,0.833333,0.000000,0.000000,0.000000,0.407962,...,0.000000,0.000000,0.25,0.0,0.5,0.0,0.0,0.0,0.0,0.0
1456,0.000000,0.219178,0.055505,0.555556,0.625,0.633333,0.074375,0.139972,0.110583,0.252140,...,0.000000,0.333333,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1457,0.294118,0.154110,0.036187,0.666667,1.000,0.933333,0.000000,0.048724,0.000000,0.375428,...,0.333333,0.000000,0.00,0.0,0.5,0.0,0.0,0.0,0.0,0.0
1458,0.000000,0.160959,0.039342,0.444444,0.625,0.766667,0.000000,0.008682,0.698100,0.000000,...,0.333333,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
from sklearn.metrics import r2_score

In [31]:
X=df_train
Y=output_col

In [32]:
final_accuracies={}
algorithms_tested=["LinearRegression()","RandomForestRegressor()","SVR()","KNeighborsRegressor()","SGDRegressor()","DecisionTreeRegressor()","ElasticNet()","Lasso()","Ridge()"]
for al in algorithms_tested:
    final_accuracies[al]=0

In [33]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=None,shuffle=True)


Here we are finally using Sequential Feature Selector from mlxtend library.It sequentially takes features, one by one and adds into our final model so to check which feature set gives us best score. It continuosuly checks r2_score whenever a feature is added. This is done so that we can get a subset of features which give us highest r2_score and remove the unwanted or noisy columns.


The minimum features we have selected as 3 . We have also set the floating parameter as True since the add an additional exclusion or inclusion step to remove features once they were included (or excluded) so that a larger number of feature subset combinations can be sampled.  

In [34]:
def sf_selector(algo:str,forward:bool,floating:bool,scoring:int,X_train,Y_train,X_test,Y_test,min_featues:int,cv:int,verbose:int):
    model=eval(algo)
    sfs=SFS(model, 
           k_features=(min_featues,len(X_train.columns)), 
           forward=forward, 
           floating=floating, 
           verbose=verbose,
           scoring=scoring,
           cv=cv,
           n_jobs=-1)
    sfs=sfs.fit(X_train, Y_train)
    print("Selected Features",sfs.k_feature_names_)
    model_=eval(algo)
    model_.fit(X_train.loc[:,list(sfs.k_feature_names_)],Y_train)
    score=model_.score(X_test.loc[:,list(sfs.k_feature_names_)],Y_test)
    
    print(f"Prediction using selected features using {model_} is :",score)
   
        
    final_accuracies[algo]=score


In [35]:
sf_selector(algo="LinearRegression()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  64 out of  71 | elapsed:    5.0s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  71 out of  71 | elapsed:    5.0s finished

[2023-02-07 18:49:43] Features: 1/71 -- score: 0.6252431837191337[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:    0.4s finished

[2023-02-07 18:49:44] Features: 2/71 -- score: 0.6860089682160042[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  69 out of  69 | elapsed:    0.4s finished

[2023-02-07 18:49:44] Features: 3/71 -- score: 0.7244031818856098[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  68 out of  68 | elapsed:    0.3s finished

[2023-02-07 18:49:45] Features: 4/71 -- score: 0.737831023837592[Paral

[Parallel(n_jobs=-1)]: Done  34 out of  34 | elapsed:    0.4s finished

[2023-02-07 18:50:02] Features: 38/71 -- score: 0.8044840248717484[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 out of  33 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  33 out of  33 | elapsed:    0.4s finished

[2023-02-07 18:50:02] Features: 39/71 -- score: 0.8044072674734999[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  32 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  32 out of  32 | elapsed:    0.4s finished

[2023-02-07 18:50:03] Features: 40/71 -- score: 0.8042555501267479[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  31 out of  31 | elapsed:    0.4s finished

[2023-02-07 18:50:03] Features: 41/71 -- score: 0.8040688445907431[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 co

Selected Features ('MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'MasVnrArea', '1stFlrSF', '2ndFlrSF', 'BsmtFullBath', 'Fireplaces', 'GarageYrBlt', 'GarageArea', '3SsnPorch', 'ScreenPorch', 'MSZoning', 'Neighborhood', 'Condition1', 'HouseStyle', 'RoofStyle', 'ExterQual', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'KitchenQual', 'Functional', 'GarageFinish', 'PavedDrive', 'SaleType')
Prediction using selected features using LinearRegression() is : 0.8584674417674243


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished

[2023-02-07 18:50:13] Features: 69/71 -- score: -2.471609056140461e+23[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.0s finished

[2023-02-07 18:50:13] Features: 70/71 -- score: -8.068447140661996e+23[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished

[2023-02-07 18:50:13] Features: 71/71 -- score: -5.326579506783321e+21

In [36]:
sf_selector(algo="RandomForestRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done  71 out of  71 | elapsed:   26.4s finished

[2023-02-07 18:50:39] Features: 1/71 -- score: 0.6605375949101939[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:   26.2s finished

[2023-02-07 18:51:06] Features: 2/71 -- score: 0.7369648266404004[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   17.2s
[Parallel(n_jobs=-1)]: Done  69 out of  69 | elapsed:   32.3s finished

[2023-02-07 18:51:38] Features: 3/71 -- score: 0.76811899838755[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   20.0s
[Parallel(n_jobs=-1)]: Done  68 out of 

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  41 out of  41 | elapsed:  1.3min finished

[2023-02-07 19:24:11] Features: 31/71 -- score: 0.8671440428229916[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.5min finished

[2023-02-07 19:25:44] Features: 32/71 -- score: 0.8680069581736843[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  39 out of  39 | elapsed:  1.1min finished

[2023-02-07 19:26:51] Features: 33/71 -- score: 0.865682953930832[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 out of  38 | elapsed:  1.2min finished

[2023-02-07 19:28:03] Features: 34/71 -- score: 0.8655923159386338[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   16.7s finished

[2023-02-07 19:51:10] Features: 67/71 -- score: 0.8513621765667286[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   10.6s finished

[2023-02-07 19:51:21] Features: 68/71 -- score: 0.8526476831305377[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    9.0s finished

[2023-02-07 19:51:30] Features: 69/71 -- score: 0.8485985825211554[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    8.4s finished

[2023-02-07 19:51:38] Features: 70/71 -- score: 0.8457806616634906[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    6.6s finished

[2023-02-07 19:51:45] Features: 71/71 -- score: 0.8425874049403339

Selected Features ('MSSubClass', 'LotArea', 'OverallQual', 'YearRemodAdd', 'BsmtFinSF1', '1stFlrSF', '2ndFlrSF', 'BsmtFullBath', 'HalfBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'Utilities', 'Neighborhood', 'BldgType', 'HouseStyle', 'Exterior1st', 'Foundation', 'BsmtExposure', 'BsmtFinType1', 'HeatingQC', 'CentralAir', 'Electrical', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive')
Prediction using selected features using RandomForestRegressor() is : 0.8849031590789184


In [37]:
sf_selector(algo="SVR()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done  71 out of  71 | elapsed:   11.5s finished

[2023-02-07 19:51:58] Features: 1/71 -- score: -0.04888744560241252[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:   12.0s finished

[2023-02-07 19:52:10] Features: 2/71 -- score: -0.04899415335655348[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done  69 out of  69 | elapsed:   12.2s finished

[2023-02-07 19:52:22] Features: 3/71 -- score: -0.04907332059071749[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done  68

[Parallel(n_jobs=-1)]: Done  42 out of  42 | elapsed:    9.4s finished

[2023-02-07 19:57:23] Features: 30/71 -- score: -0.050533013450070594[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done  41 out of  41 | elapsed:    9.4s finished

[2023-02-07 19:57:32] Features: 31/71 -- score: -0.05055585217120808[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    8.2s finished

[2023-02-07 19:57:41] Features: 32/71 -- score: -0.050574950449314836[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  39 out of  39 | elapsed:    8.0s finished

[2023-02-07 19:57:49] Features: 33/71 -- score: -0.0505966929185222[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n

[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    0.9s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    1.5s finished

[2023-02-07 20:01:26] Features: 66/71 -- score: -0.05139591845039049[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.4s finished

[2023-02-07 20:01:28] Features: 67/71 -- score: -0.05142025957700573[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.9s finished

[2023-02-07 20:01:29] Features: 68/71 -- score: -0.051451043294565534[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.8s finished

[2023-02-07 20:01:30] Features: 69/71 -- score: -0.051486184737385844[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:

Selected Features ('LotArea', 'MiscVal', 'ExterQual')
Prediction using selected features using SVR() is : -0.04220340910328968


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.5s finished

[2023-02-07 20:01:31] Features: 71/71 -- score: -0.05157369931969793

In [38]:
sf_selector(algo="KNeighborsRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  71 out of  71 | elapsed:    0.6s finished

[2023-02-07 20:01:32] Features: 1/71 -- score: 0.5952105986886512[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:    0.5s finished

[2023-02-07 20:01:33] Features: 2/71 -- score: 0.7118988236817737[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  69 out of  69 | elapsed:    0.6s finished

[2023-02-07 20:01:33] Features: 3/71 -- score: 0.7907465587570103[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  68 out of  68 | elapsed:    0.6s finished

[2023-02-07 20:01:34] Features: 4/71 -- score: 0.7973875537077513[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  67 out of  67 | elapsed:    0.6s finished



[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    0.7s finished

[2023-02-07 20:02:03] Features: 36/71 -- score: 0.7739795457618126[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  35 | elapsed:    0.6s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    0.7s finished

[2023-02-07 20:02:04] Features: 37/71 -- score: 0.7745305569576941[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 out of  34 | elapsed:    0.6s finished

[2023-02-07 20:02:05] Features: 38/71 -- score: 0.7758634217011708[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 out of  33 | elapsed:    0.5s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  33 out of  33 | elapsed:    0.6s finished

[2023-02-07 20:02:05] Features: 39/71 -- score: 0.7740466302746647[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 co

Selected Features ('LotArea', 'OverallQual', 'BsmtFinSF1', '1stFlrSF', '2ndFlrSF', 'KitchenAbvGr', 'GarageYrBlt', 'MiscVal', 'MSZoning', 'Street', 'Utilities', 'Condition2', 'RoofMatl', 'Heating', 'Electrical')
Prediction using selected features using KNeighborsRegressor() is : 0.8407274902887504


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished

[2023-02-07 20:02:18] Features: 69/71 -- score: 0.7284066908353232[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.0s finished

[2023-02-07 20:02:18] Features: 70/71 -- score: 0.7221439723167254[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished

[2023-02-07 20:02:18] Features: 71/71 -- score: 0.7052907442743178

In [39]:
sf_selector(algo="DecisionTreeRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  71 out of  71 | elapsed:    0.3s finished

[2023-02-07 20:02:19] Features: 1/71 -- score: 0.6591484536282441[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:    0.3s finished

[2023-02-07 20:02:19] Features: 2/71 -- score: 0.7196226322716497[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  69 out of  69 | elapsed:    0.4s finished

[2023-02-07 20:02:20] Features: 3/71 -- score: 0.7297039328443882[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  68 out of  68 | elapsed:    0.5s finished

[2023-02-07 20:02:20] Features: 4/71 -- score: 0.7353019339528141[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  67 out of  67 | elapsed:    0.4s finished



[Parallel(n_jobs=-1)]: Done  34 out of  34 | elapsed:    0.8s finished

[2023-02-07 20:02:47] Features: 38/71 -- score: 0.7714168107962406[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 out of  33 | elapsed:    0.7s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  33 out of  33 | elapsed:    0.8s finished

[2023-02-07 20:02:48] Features: 39/71 -- score: 0.7788663407819163[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  32 out of  32 | elapsed:    0.8s finished

[2023-02-07 20:02:49] Features: 40/71 -- score: 0.7641134420726731[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  31 out of  31 | elapsed:    0.8s finished

[2023-02-07 20:02:50] Features: 41/71 -- score: 0.7780409113166472[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.8s f

Selected Features ('OverallQual', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'EnclosedPorch', '3SsnPorch', 'PoolArea', 'MiscVal', 'YrSold', 'MSZoning', 'Street', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'MasVnrType', 'Foundation', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'Heating', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition')
Prediction using selected features using DecisionTreeRegressor() is : 0.6722766428095865


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished

[2023-02-07 20:03:08] Features: 71/71 -- score: 0.66371218296074

In [40]:
sf_selector(algo="SGDRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  64 out of  71 | elapsed:    1.9s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  71 out of  71 | elapsed:    2.0s finished

[2023-02-07 20:03:10] Features: 1/71 -- score: 0.624798545688835[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:    5.1s finished

[2023-02-07 20:03:15] Features: 2/71 -- score: 0.6853375502809435[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  69 out of  69 | elapsed:    5.5s finished

[2023-02-07 20:03:21] Features: 3/71 -- score: 0.7211796631422368[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1

[Parallel(n_jobs=-1)]: Done  42 out of  42 | elapsed:    4.7s finished

[2023-02-07 20:06:23] Features: 30/71 -- score: 0.8003265732985311[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  41 out of  41 | elapsed:    4.7s finished

[2023-02-07 20:06:28] Features: 31/71 -- score: 0.8001918726771328[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    4.6s finished

[2023-02-07 20:06:33] Features: 32/71 -- score: 0.7999852675989585[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  39 out of  39 | elapsed:    4.6s finished

[2023-02-07 20:06:38] Features: 33/71 -- score: 0.8004887969078348[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)

[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    0.3s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.6s finished

[2023-02-07 20:07:49] Features: 66/71 -- score: 0.7923170966807904[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.5s finished

[2023-02-07 20:07:49] Features: 67/71 -- score: 0.7919374050339337[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.4s finished

[2023-02-07 20:07:50] Features: 68/71 -- score: 0.7932711760881834[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.3s finished

[2023-02-07 20:07:50] Features: 69/71 -- score: 0.7900017122992671[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.2s f

Selected Features ('MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'MasVnrArea', '1stFlrSF', '2ndFlrSF', 'BsmtFullBath', 'GarageArea', 'MiscVal', 'MSZoning', 'LandContour', 'Neighborhood', 'Condition1', 'BldgType', 'ExterQual', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageFinish', 'GarageCond')
Prediction using selected features using SGDRegressor() is : 0.8160338662638307


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.2s finished

[2023-02-07 20:07:51] Features: 71/71 -- score: 0.7788008076513788

In [41]:
sf_selector(algo="ElasticNet()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  71 out of  71 | elapsed:    0.3s finished

[2023-02-07 20:07:51] Features: 1/71 -- score: 0.12517535989289047[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:    0.3s finished

[2023-02-07 20:07:51] Features: 2/71 -- score: 0.1859361085152479[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  69 out of  69 | elapsed:    0.3s finished

[2023-02-07 20:07:52] Features: 3/71 -- score: 0.22275816434252266[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  68 out of  68 | elapsed:    0.3s finished

[2023-02-07 20:07:52] Features: 4/71 -- score: 0.2512463775775771[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  67 out of  67 | elapsed:    0.3s finished

[Parallel(n_jobs=-1)]: Done  34 out of  34 | elapsed:    0.3s finished

[2023-02-07 20:08:05] Features: 38/71 -- score: 0.44928870757894723[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 out of  33 | elapsed:    0.3s finished

[2023-02-07 20:08:06] Features: 39/71 -- score: 0.4495605673587873[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  32 out of  32 | elapsed:    0.2s finished

[2023-02-07 20:08:06] Features: 40/71 -- score: 0.4498144990622085[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  31 out of  31 | elapsed:    0.3s finished

[2023-02-07 20:08:06] Features: 41/71 -- score: 0.4500577073670696[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.2s 

Selected Features ('MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', '1stFlrSF', '2ndFlrSF', 'BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'ScreenPorch', 'MiscVal', 'MoSold', 'YrSold', 'MSZoning', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'BldgType', 'HouseStyle', 'RoofStyle', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageCond', 'PavedDrive', 'SaleType')
Prediction using selected features using ElasticNet() is : 0.46665767507852673


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.0s finished

[2023-02-07 20:08:13] Features: 70/71 -- score: 0.45054449361738963[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished

[2023-02-07 20:08:13] Features: 71/71 -- score: 0.4504209034657303

In [42]:
sf_selector(algo="Lasso()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  71 out of  71 | elapsed:    0.3s finished

[2023-02-07 20:08:14] Features: 1/71 -- score: 0.6252448627722356[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  63 out of  70 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:    0.3s finished

[2023-02-07 20:08:14] Features: 2/71 -- score: 0.6860127612274918[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  69 out of  69 | elapsed:    0.3s finished

[2023-02-07 20:08:14] Features: 3/71 -- score: 0.724409734636117[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  61 out of  68 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  68 out of  68 | elapsed:    0.3s finished

[2023-02-07 20:08:15] Features: 4/71 -- score: 0.737

[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    0.3s finished

[2023-02-07 20:08:28] Features: 37/71 -- score: 0.8046634829016576[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 out of  34 | elapsed:    0.3s finished

[2023-02-07 20:08:29] Features: 38/71 -- score: 0.804502767834135[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 out of  33 | elapsed:    0.3s finished

[2023-02-07 20:08:29] Features: 39/71 -- score: 0.8044278906193509[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  32 out of  32 | elapsed:    0.3s finished

[2023-02-07 20:08:30] Features: 40/71 -- score: 0.8042790796921093[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  31 out of  31 | elapsed:    0.3s finished

[2023-02-07 20:08:30] Features: 41/71 -- score: 0.8041040848057289[Parall

Selected Features ('MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'MasVnrArea', '1stFlrSF', '2ndFlrSF', 'BsmtFullBath', 'Fireplaces', 'GarageYrBlt', 'GarageArea', '3SsnPorch', 'ScreenPorch', 'MSZoning', 'Neighborhood', 'Condition1', 'HouseStyle', 'RoofStyle', 'ExterQual', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'KitchenQual', 'Functional', 'GarageFinish', 'PavedDrive', 'SaleType')
Prediction using selected features using Lasso() is : 0.8584248132533064


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished

[2023-02-07 20:08:38] Features: 71/71 -- score: 0.7214683712581473

In [43]:
sf_selector(algo="Ridge()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  71 out of  71 | elapsed:    0.3s finished

[2023-02-07 20:08:39] Features: 1/71 -- score: 0.6248451357260899[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:    0.3s finished

[2023-02-07 20:08:39] Features: 2/71 -- score: 0.6860187405545658[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  69 out of  69 | elapsed:    0.3s finished

[2023-02-07 20:08:39] Features: 3/71 -- score: 0.72454488762339[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  68 out of  68 | elapsed:    0.3s finished

[2023-02-07 20:08:40] Features: 4/71 -- score: 0.7378764575252801[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  67 out of  67 | elapsed:    0.3s finished

[2

[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    0.3s finished

[2023-02-07 20:08:52] Features: 37/71 -- score: 0.8057380730255164[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 out of  34 | elapsed:    0.3s finished

[2023-02-07 20:08:53] Features: 38/71 -- score: 0.8056668523566797[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 out of  33 | elapsed:    0.3s finished

[2023-02-07 20:08:53] Features: 39/71 -- score: 0.8055796148514341[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  32 out of  32 | elapsed:    0.2s finished

[2023-02-07 20:08:53] Features: 40/71 -- score: 0.8057767751686102[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  31 out of  31 | elapsed:    0.2s finished

[2023-02-07 20:08:54] Features: 41/71 -- score: 0.8057308045361491[Paral

Selected Features ('MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearRemodAdd', 'MasVnrArea', '1stFlrSF', '2ndFlrSF', 'BsmtFullBath', 'Fireplaces', 'GarageYrBlt', 'GarageArea', '3SsnPorch', 'ScreenPorch', 'MSZoning', 'Neighborhood', 'Condition1', 'RoofStyle', 'ExterQual', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'KitchenQual', 'Functional', 'GarageFinish', 'PavedDrive', 'SaleType')
Prediction using selected features using Ridge() is : 0.855075595056031


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished

[2023-02-07 20:09:00] Features: 69/71 -- score: 0.7848729458072176[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.0s finished

[2023-02-07 20:09:00] Features: 70/71 -- score: 0.7749825171269353[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished

[2023-02-07 20:09:00] Features: 71/71 -- score: 0.7589919079741888

In [44]:
final_accuracies

{'LinearRegression()': 0.8584674417674243,
 'RandomForestRegressor()': 0.8849031590789184,
 'SVR()': -0.04220340910328968,
 'KNeighborsRegressor()': 0.8407274902887504,
 'SGDRegressor()': 0.8160338662638307,
 'DecisionTreeRegressor()': 0.6722766428095865,
 'ElasticNet()': 0.46665767507852673,
 'Lasso()': 0.8584248132533064,
 'Ridge()': 0.855075595056031}